In [4]:
!pip install -q transformers torch accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.8 MB/s eta 0:00:00


In [5]:
import torch
from transformers import AutoTokenizer, GPT2LMHeadModel

# Ödev tanımına ve parametre sayısına en uygun model
model_id = "ytu-ce-cosmos/turkish-gpt2-medium"

# Tokenizer yükleme ve Padding ayarı (Hücre 3'ün patlamaması için kritik)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# Modeli GPU'ya yükleme
device = "cuda" if torch.cuda.is_available() else "cpu"
model = GPT2LMHeadModel.from_pretrained(model_id).to(device)

print(f"Model başarıyla {device} üzerinde yüklendi!")

Model başarıyla cuda üzerinde yüklendi!


In [7]:
# Örnek latex kodları. matematiksel ifade içermiyorlar sözel sorular.
# Ama yine de latex syntaxı içeriyor.


import json

sozel_sorular = [
    {"soru": "Ali'nin ismini matematiksel bir küme içerisinde $\\mathbf{A}$ harfiyle göster ve 'Ali' metnini $\\text{italik}$ yazdır.", "tip": "kume"},
    {"soru": "'Elma' ve 'Armut' kelimelerinin kesişimini LaTeX kesişim sembolü kullanarak yaz.", "tip": "kesisim"},
    {"soru": "'Hız' kelimesinin altına 'ortalama' yazacak şekilde bir indis (subscript) LaTeX kodu yaz.", "tip": "indis"},
    {"soru": "Bir cümlenin başına 'Dikkat' kelimesini $\\boxed{\\text{Dikkat}}$ içerisine alarak yaz.", "tip": "kutu"},
    {"soru": "'Gökhan' isminin üzerine bir 'şapka' (hat) sembolü ekle.", "tip": "sapka"},
    {"soru": "'Mavi' ve 'Yeşil' renklerini LaTeX birleşim sembolü ile birbirine bağla.", "tip": "birlesim"},
    {"soru": "'Sıcaklık' kelimesini derece sembolü ile birlikte yaz.", "tip": "derece"},
    {"soru": "'Doğru' kelimesinin yanına 'Yanlış' değil anlamında bir 'eşit değildir' ($\\neq$) işareti koy.", "tip": "esit_degil"},
    {"soru": "'Kitap' kelimesini $\\overline{\\text{üzeri çizgili}}$ şekilde yazdır.", "tip": "cizgi"},
    {"soru": "'X' ve 'Y' değişkenlerini bir kesir çizgisi kullanarak, X üstte Y altta olacak şekilde göster.", "tip": "kesir"}
]

with open('sozel_test_verisi.json', 'w', encoding='utf-8') as f:
    json.dump(sozel_sorular, f, ensure_ascii=False, indent=4)




The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


--- SÖZEL LATEX TESTİ BAŞLIYOR ---

Test 1 [kume]:
Soru: Ali'nin ismini matematiksel bir küme içerisinde $\mathbf{A}$ harfiyle göster ve 'Ali' metnini $\text{italik}$ yazdır.
Model: Ali'nin ismini matematiksel bir küme içerisinde $\mathbf{A}$ harfiyle göster ve 'Ali' metnini $\text{italik}$ yazdır.
Ali'nin ismini matematiksel bir küme içerisinde $\mathbf{A}$ harfiyle göster ve 'Ali' metnini $\text{italik}$ yazdır.
Ali'nin ismini matematiksel bir küme içerisinde $\mathbf{A}$ harfiyle göster ve 'Ali
--------------------------------------------------
Test 2 [kesisim]:
Soru: 'Elma' ve 'Armut' kelimelerinin kesişimini LaTeX kesişim sembolü kullanarak yaz.
Model: 'Elma' ve 'Armut' kelimelerinin kesişimini LaTeX kesişim sembolü kullanarak yaz.
LaTeX kesişim sembolü, LaTeX sembolünün kesişimini temsil eder. LaTeX sembolünün kesişimini LaTeX sembolünün kesişimine dönüştürmek için LaTeX sembolünün kesişimini LaTeX sembolünün kesişimine dönüştürün.
LaTeX sembolünün kesişimini LaTeX sembolünün kes

In [9]:
# Modelin toplam parametre sayısını hesaplayan fonksiyon
def count_parameters(model):
    # 'requires_grad' olanlar eğitilebilir, olmayanlar dondurulmuş parametrelerdir
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"Toplam Parametre: {total_params:,}")
    print(f"Eğitilebilir Parametre: {trainable_params:,}")
    return total_params

count_parameters(model)

Toplam Parametre: 354,823,168
Eğitilebilir Parametre: 354,823,168


354823168

In [2]:
# TOKENIZER TEST
test_sayi = "150"
tokens = tokenizer.tokenize(test_sayi)
print(tokens)

['150']


In [6]:
import torch

def cosmos_chat():
    print("--- COSMOS (HAM MODEL) TESTİ ---")
    print("Çıkmak için 'exit' yaz.")

    while True:
        user_input = input("Sen: ")
        if user_input.lower() == 'exit':
            break

        # Base model olduğu için ona bir 'başlangıç' vermemiz lazım
        prompt = f"### Soru: {user_input}\n### Cevap:"

        inputs = tokenizer(prompt, return_tensors="pt").to(device)

        # Modeli serbest bırakıyoruz (temperature'ı artırdık ki daha çok saçmalasın)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=150,
                temperature=0.8, # Daha yaratıcı (!) saçmalaması için
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )

        cevap = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Cevabı temizle
        try:
            model_cikti = cevap.split("### Cevap:")[1].strip()
        except:
            model_cikti = cevap # Eğer formatı bozarsa her şeyi bas

        print(f"Cosmos: {model_cikti}")
        print("-" * 30)

cosmos_chat()

--- COSMOS (HAM MODEL) TESTİ ---
Çıkmak için 'exit' yaz. Bakalım ne kadar saçmalayacak...

Sen: bugün hava kaç derece
Cosmos: hava sıcaklığı 6 derece
### Soru: bugün hava sıcaklığı 6 derece nasıl
------------------------------
Sen: domates hi renkngş
Cosmos: domates hi renkngş
domates hi renkngş
Knight Of Tsushima'nın Yeni Haritası - Haberler - Türk Anime TV
Knight Of Tsushima'nın Yeni Haritası
[img]https://www.youtube.com/watch?v=Kd0TZxK_3Rs[/img]
[img]https://www.youtube.com/watch?v=JzgCpcAWkU[/img]
[img]https://www.youtube.com/watch?v=S_kXwK1c7Vz[/img]
[img]https://www.youtube.com/watch?
------------------------------
Sen: domates hangi renk
Cosmos: domates hangi renk
##1 Merhaba canan hanım domateslerin sapları beyazdır daha cok beyaz rengi vardır. Domateslerin sapları beyaz ise hangi renk olduğunu nasıl anlarız?
#2 Değerli maddemiz demirdir bu demirin rengi de beyazdır.
#3 Değerli maddemizin rengi sarıdır.
#4 Değerli maddemizin rengi kırmızıdır.
#5 Değerli maddemizin rengi turuncu


KeyboardInterrupt



In [ ]:
#DOSYA YUKLEME PENCERESİ
from google.colab import files

# Bu satırı çalıştırdığında önüne dosya seçme penceresi açılacak
uploaded = files.upload()

# Yüklediğin dosyanın adını alalım
for filename in uploaded.keys():
    print(f'"{filename}" başarıyla yüklendi!')
    # Dosya artık Colab'in ana dizininde, yani yol (path) sadece dosya adı oldu.

In [ ]:
#MOUNT

import json
from datasets import Dataset
from google.colab import drive

# 1. Drive'ı bağla (Eğer daha önce bağlamadıysan)
drive.mount('/content/drive')

# 2. Dosya yolunu tam olarak senin dosya ismine göre ayarla
# Not: Dosyan Drive'ın ana dizinindeyse bu yol doğrudur.
# Bir klasör içindeyse araya klasör ismini ekle.
file_path = '/content/drive/MyDrive/AAAAA.json'  # --> ISMINI INDIRDIGIN JSONLA DEGSTOR

with open(file_path, 'r', encoding='utf-8') as f:
    # JSON dosyası liste formatındaysa direkt okur
    raw_data = json.load(f)

# Veri setindeki anahtarları (key) kontrol etmemiz lazım.
# Eğer hata verirse 'question' yerine 'instruction' yazacağız.
dataset = Dataset.from_list(raw_data)
print(f"Başarıyla yüklendi! Örnek sayısı: {len(dataset)}")

In [ ]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from transformers import TrainingArguments

def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['text'])):
        # Modelin "Asistan" olduğunu anlaması için Soru ve Cevap ayrılmalı
        # Bilgem verisi açıklayıcı olduğu için soruyu ona göre uyduruyoruz
        text = (
            f"### Soru: Lütfen şu konuyu matematiksel olarak açıkla: {example['text'][i][:50]}...\n" # Konu başlığı gibi
            f"### Cevap: {example['text'][i]}{tokenizer.eos_token}"
        )
        output_texts.append(text)
    return output_texts

# KRİTİK DEĞİŞİKLİK: Model '### Cevap:' görene kadar olan kısmı (soruyu) maskeler.
# Böylece soruyu ezberlemez, sadece cevap vermeyi öğrenir.
response_template = "### Cevap:"
collator = DataCollatorForCompletionOnlyLM(response_template=response_template, tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/cosmos_final",
    per_device_train_batch_size=1,      # 350M model için 2-4 yapılabilir (VRAM'e göre)
    gradient_accumulation_steps=8,      # 2*4 = 8 effective batch size
    learning_rate=2e-5,                 # Biraz daha iddialı bir hız
    num_train_epochs=1,                 # 400MB veri için 1 epoch az gelebilir, 2 idealdir
    fp16=True,
    logging_steps=5,
    warmup_ratio=0.1,                   # Başta modelin sarsılmaması için yavaş başla
    save_strategy="no",
    optim="adamw_torch"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    max_seq_length=1024,                # Bilgem metinleri uzun, 512 yetmeyebilir, 1024 yapalım
    args=training_args,
)

trainer.train()